In [5]:
from urllib.request import urlopen, Request
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()
user_agent = os.environ['USER_AGENT']

In [7]:
headers = {
    'authority': 'finviz.com',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'upgrade-insecure-requests': '1',
    'user-agent': user_agent,
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'nl-BE,nl-NL;q=0.9,nl;q=0.8,en-US;q=0.7,en;q=0.6'}

In [8]:
'''loop to get full table per ticker'''
news_tables = {}
financial = {}
tickers = ['TDOC','RDS-A', 'EURN', 'V', 'PYPL', 'ABBV', 'ADBE', 'AMGN', 'BABA', 'CVS', 'MO', 'MSFT', 'PLTR', 'SBS', 'TTE', 'HSI', 'CRM', 'CRSP' ]

for ticker in tickers:
    print(f"sending request for {ticker}")
    params = (
    ('t', ticker),
    )
    response = requests.get('https://finviz.com/quote.ashx', headers=headers, params=params) 
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response.content, "html.parser")
    try:
        # Find 'news-table' in the Soup and load it into 'news_table'
        news_table = html.find('table', class_='fullview-news-outer').find_all('tr')
        snapshot_table = html.find('table', class_='snapshot-table2')
        # Add the table to our dictionary
        news_tables[ticker] = news_table
        financial[ticker] = snapshot_table
    except AttributeError:
        pass

sending request for TDOC
sending request for RDS-A
sending request for EURN
sending request for V
sending request for PYPL
sending request for ABBV
sending request for ADBE
sending request for AMGN
sending request for BABA
sending request for CVS
sending request for MO
sending request for MSFT
sending request for PLTR
sending request for SBS
sending request for TTE
sending request for HSI
sending request for CRM
sending request for CRSP


In [9]:
news_tables.keys()

dict_keys(['TDOC', 'RDS-A', 'EURN', 'V', 'PYPL', 'ABBV', 'ADBE', 'AMGN', 'BABA', 'CVS', 'MO', 'MSFT', 'PLTR', 'SBS', 'TTE', 'CRM', 'CRSP'])

In [10]:
'''Get final dict with news'''
new_dict = {}

# Iterate through the news
for file_name, news_table in news_tables.items():
    # Create new list per iteration
    parsed_news = []
    intermediate_dict = {}
    # Iterate through all tr tags in 'news_table'
    for x in news_table:
        # read the text from each tr tag into text
        # get text from a only
        text = x.a.get_text() 
        # splite text in the td tag into a list 
        date_scrape = x.td.text.split()
        # if the length of 'date_scrape' is 1, load 'time' as the only element

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        # else load 'date' as the 1st element and 'time' as the second    
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        # Extract the ticker from the file name, get the string up to the 1st '_'  
        #ticker = file_name.split('_')[0]
        
        # Append ticker, date, time and headline as a list to the 'parsed_news' list
        parsed_news.append([date, text])
    #intermediate dictionary
    intermediate_dict['news'] = parsed_news
    # Append full list to dict
    new_dict[file_name] = intermediate_dict


In [11]:
'''Get financial dict'''
financial_dict = {}
for ticker, values in financial.items():
    table_dark_row = values.find_all('tr', class_="table-dark-row")
    values_dict = {}
    intermediate_dict = {}
    #table_dark_row = financial['TDOC'].find_all('tr', class_="table-dark-row")
    for i in table_dark_row:
        keys = i.find_all('td', class_="snapshot-td2-cp")
        values = i.find_all('td', class_="snapshot-td2")
        for keys, values in zip(keys, values):
            values_dict[keys.text] =  values.text
    intermediate_dict['fundamentals'] = values_dict
    new_dict[ticker]['fundamentals'] = values_dict


In [12]:
'''Clean data'''
def clean_data(data):
    #filling empty cells with nan value and treating '52W Range' column
    #mymap = {'-': float('nan')}
    range_df = data['52W Range']
    data.drop(columns=['52W Range'], axis=1, inplace=True)
    #data.applymap(lambda s: mymap.get(s) if s in mymap else s)
    
    range_df = range_df.apply(lambda x: x.split(' - '))
    data['52W Range Low'] = range_df.apply(lambda x: x[0])
    data['52W Range High'] = range_df.apply(lambda x: x[1])
    #treating % and Million/Billion values to numeric
    mymap = {'%': 0.01, 'M': 1000000, 'B': 1000000000}
    metrics_tonumeric = ['Market Cap', 'Dividend %', 'Shs Outstand']
    for metric in metrics_tonumeric:
        data.loc[:, metric] = pd.to_numeric(data[metric].str[:-1]) * \
                                    data[metric].str[-1].replace(mymap)
    metrics_tonumeric2 = ['EPS next 5Y', 'Insider Own', 
                         'ROE', 'ROI', 'ROA', 'Profit Margin']
    empty_metric = []
    for metric in metrics_tonumeric2:
        empty_metric.append(f"{metric} %")
    
    for new, old in zip(empty_metric, metrics_tonumeric2):
        data[new] = pd.to_numeric(data[old].str[:-1]) * \
                                    data[old].str[-1].replace(mymap)
    #transforming whole df to numeric values
    #data = data.apply(pd.to_numeric, errors='coerce')
    #adjustments on dataframe to make it more readable
    data['Market Cap(B)'] = data['Market Cap'].apply(lambda x: x/1000000000)
    data['Shs Outstand(M)'] = data['Shs Outstand'].apply(lambda x: x/1000000)
    data.replace('-', np.nan, inplace=True)
    columns = ['Market Cap(B)', 'Price', 'Target Price','P/B', 'P/E', 'Forward P/E', 'P/S', 'PEG', 
                         'P/FCF', 'Debt/Eq', 'EPS (ttm)', 'EPS next 5Y %', 'Dividend %', 
                         'Insider Own %', 'ROE %', 'ROI %', 'ROA %', 'Profit Margin %', 
                         'Shs Outstand(M)', 'RSI (14)', 'Beta', '52W Range Low', '52W Range High']
    return data[columns].astype(float)

In [13]:
new_dict['V']['fundamentals']

{'Index': 'DJIA S&P500',
 'P/E': '50.77',
 'EPS (ttm)': '4.18',
 'Insider Own': '0.13%',
 'Shs Outstand': '1.98B',
 'Perf Week': '-3.71%',
 'Market Cap': '447.97B',
 'Forward P/E': '30.10',
 'EPS next Y': '20.89%',
 'Insider Trans': '-5.96%',
 'Shs Float': '1.84B',
 'Perf Month': '-8.09%',
 'Income': '10.34B',
 'PEG': '2.82',
 'EPS next Q': '1.70',
 'Inst Own': '96.70%',
 'Short Float': '1.51%',
 'Perf Quarter': '-9.48%',
 'Sales': '21.35B',
 'P/S': '20.98',
 'EPS this Y': '-18.60%',
 'Inst Trans': '-0.03%',
 'Short Ratio': '3.26',
 'Perf Half Y': '-5.88%',
 'Book/sh': '17.54',
 'P/B': '12.10',
 'ROA': '12.80%',
 'Target Price': '275.42',
 'Perf Year': '0.86%',
 'Cash/sh': '9.12',
 'P/C': '23.28',
 'EPS next 5Y': '17.97%',
 'ROE': '30.80%',
 '52W Range': '192.81 - 252.67',
 'Perf YTD': '-2.94%',
 'Dividend': '1.28',
 'P/FCF': '52.31',
 'EPS past 5Y': '10.90%',
 'ROI': '20.20%',
 '52W High': '-15.98%',
 'Beta': '0.94',
 'Dividend %': '0.60%',
 'Quick Ratio': '2.00',
 'Sales past 5Y': '9

In [14]:
np.__version__, pd.__version__

('1.21.4', '1.3.4')

In [15]:
new_dict['V']['fundamentals']

{'Index': 'DJIA S&P500',
 'P/E': '50.77',
 'EPS (ttm)': '4.18',
 'Insider Own': '0.13%',
 'Shs Outstand': '1.98B',
 'Perf Week': '-3.71%',
 'Market Cap': '447.97B',
 'Forward P/E': '30.10',
 'EPS next Y': '20.89%',
 'Insider Trans': '-5.96%',
 'Shs Float': '1.84B',
 'Perf Month': '-8.09%',
 'Income': '10.34B',
 'PEG': '2.82',
 'EPS next Q': '1.70',
 'Inst Own': '96.70%',
 'Short Float': '1.51%',
 'Perf Quarter': '-9.48%',
 'Sales': '21.35B',
 'P/S': '20.98',
 'EPS this Y': '-18.60%',
 'Inst Trans': '-0.03%',
 'Short Ratio': '3.26',
 'Perf Half Y': '-5.88%',
 'Book/sh': '17.54',
 'P/B': '12.10',
 'ROA': '12.80%',
 'Target Price': '275.42',
 'Perf Year': '0.86%',
 'Cash/sh': '9.12',
 'P/C': '23.28',
 'EPS next 5Y': '17.97%',
 'ROE': '30.80%',
 '52W Range': '192.81 - 252.67',
 'Perf YTD': '-2.94%',
 'Dividend': '1.28',
 'P/FCF': '52.31',
 'EPS past 5Y': '10.90%',
 'ROI': '20.20%',
 '52W High': '-15.98%',
 'Beta': '0.94',
 'Dividend %': '0.60%',
 'Quick Ratio': '2.00',
 'Sales past 5Y': '9

In [16]:
data = pd.DataFrame.from_records(new_dict['V']['fundamentals'], index=['V'],coerce_float=True)


In [17]:
data

,52W High,52W Low,52W Range,ATR,Avg Volume,Beta,Book/sh,Cash/sh,Change,Current Ratio,...,Sales Q/Q,Sales past 5Y,Short Float,Short Ratio,Shortable,Shs Float,Shs Outstand,Target Price,Volatility,Volume
V,-15.98%,10.11%,192.81 - 252.67,5.28,8.50M,0.94,17.54,9.12,0.10%,2.00,...,26.70%,9.50%,1.51%,3.26,Yes,1.84B,1.98B,275.42,2.31% 2.29%,"9,596,933"


In [18]:
df = pd.DataFrame.from_records(new_dict['V']['fundamentals'], index=['V'])
for ticker in tickers[1:]:
    try:
        df = df.append(pd.DataFrame.from_records(new_dict[ticker]['fundamentals'], index=[ticker]))
    except KeyError:
        pass

In [19]:
df

,52W High,52W Low,52W Range,ATR,Avg Volume,Beta,Book/sh,Cash/sh,Change,Current Ratio,...,Sales Q/Q,Sales past 5Y,Short Float,Short Ratio,Shortable,Shs Float,Shs Outstand,Target Price,Volatility,Volume
V,-15.98%,10.11%,192.81 - 252.67,5.28,8.50M,0.94,17.54,9.12,0.10%,2.00,...,26.70%,9.50%,1.51%,3.26,Yes,1.84B,1.98B,275.42,2.31% 2.29%,"9,596,933"
RDS-A,-10.78%,47.01%,30.80 - 50.75,1.02,5.61M,0.82,42.04,9.71,2.51%,1.30,...,36.40%,-7.40%,0.10%,0.70,Yes,3.84B,3.89B,59.08,1.80% 1.72%,"5,912,843"
EURN,-10.00%,33.51%,7.55 - 11.20,0.29,1.72M,-,11.11,-,-1.56%,-,...,-72.80%,-,1.81%,1.90,Yes,180.37M,201.68M,12.05,1.93% 2.30%,"1,175,489"
V,-15.98%,10.11%,192.81 - 252.67,5.28,8.50M,0.94,17.54,9.12,0.10%,2.00,...,26.70%,9.50%,1.51%,3.26,Yes,1.84B,1.98B,275.42,2.31% 2.29%,"9,596,933"
PYPL,-31.47%,15.80%,183.54 - 310.16,8.29,10.45M,1.13,18.82,11.66,2.04%,1.30,...,13.20%,18.30%,1.16%,1.30,Yes,1.17B,1.17B,288.75,3.55% 3.34%,"17,433,100"
ABBV,-3.86%,19.96%,97.40 - 121.53,1.65,7.22M,0.79,7.66,6.95,-0.11%,1.00,...,11.20%,14.90%,0.77%,1.89,Yes,1.77B,1.77B,127.78,1.15% 1.47%,"3,940,964"
ADBE,-2.66%,56.79%,420.78 - 677.76,12.69,2.04M,1.07,30.22,13.25,0.32%,1.40,...,22.00%,21.80%,1.05%,2.43,Yes,474.32M,477.00M,706.37,1.75% 1.97%,"1,077,608"
AMGN,-25.04%,3.46%,200.47 - 276.69,4.18,2.61M,0.61,14.49,22.92,-1.89%,1.60,...,4.40%,3.30%,2.08%,4.52,Yes,566.76M,567.00M,244.81,1.53% 1.99%,"3,129,183"
BABA,-40.65%,20.31%,138.43 - 280.61,5.36,25.36M,0.88,56.14,27.66,-0.16%,1.80,...,33.80%,48.00%,2.35%,1.86,Yes,2.00B,2.72B,243.13,2.44% 2.86%,"9,655,711"
CVS,-3.19%,43.43%,65.18 - 96.57,1.74,5.19M,0.81,56.25,9.71,-0.56%,0.90,...,10.00%,11.90%,1.22%,3.10,Yes,1.32B,1.32B,103.90,1.78% 1.85%,"3,602,100"


In [20]:
data = clean_data(df)
data

,Market Cap(B),Price,Target Price,P/B,P/E,Forward P/E,P/S,PEG,P/FCF,Debt/Eq,...,Insider Own %,ROE %,ROI %,ROA %,Profit Margin %,Shs Outstand(M),RSI (14),Beta,52W Range Low,52W Range High
V,447.97,212.30,275.42,12.10,50.77,30.10,20.98,2.82,52.31,0.61,...,0.0013,0.308,0.202,0.128,0.484,1980.00,42.61,0.94,192.81,252.67
RDS-A,177.55,45.28,59.08,1.08,38.57,7.27,0.81,4.82,9.09,0.58,...,0.0020,0.028,-0.072,0.012,0.021,3890.00,47.04,0.82,30.80,50.75
EURN,2.03,10.08,12.05,0.91,11.82,53.05,2.19,NaN,NaN,NaN,...,0.0024,NaN,NaN,NaN,NaN,201.68,46.71,NaN,7.55,11.20
V,447.97,212.30,275.42,12.10,50.77,30.10,20.98,2.82,52.31,0.61,...,0.0013,0.308,0.202,0.128,0.484,1980.00,42.61,0.94,192.81,252.67
PYPL,242.20,212.54,288.75,11.29,51.18,39.51,9.86,2.50,49.40,0.36,...,0.0010,0.239,0.084,0.068,0.201,1170.00,33.04,1.13,183.54,310.16
ABBV,205.96,116.84,127.78,15.25,27.81,8.41,3.73,6.21,16.31,5.96,...,0.0012,0.564,0.127,0.050,0.135,1770.00,65.59,0.79,97.40,121.53
ADBE,307.01,659.73,706.37,21.83,54.44,46.36,20.33,3.01,46.56,0.00,...,0.0031,0.424,0.306,0.231,0.387,477.00,58.65,1.07,420.78,677.76
AMGN,116.92,207.40,244.81,14.31,21.36,11.44,4.54,3.67,30.09,4.57,...,0.0010,0.637,0.195,0.090,0.218,567.00,42.64,0.61,200.47,276.69
BABA,455.51,166.54,243.13,2.97,19.82,2.42,3.78,1.90,16.48,0.15,...,0.1018,0.160,0.014,0.091,0.192,2720.00,51.60,0.88,138.43,280.61
CVS,123.66,93.49,103.90,1.66,16.32,11.36,0.43,2.94,9.83,0.79,...,0.0010,0.105,0.074,0.033,0.027,1320.00,64.39,0.81,65.18,96.57


In [21]:
'''
Calculation of Intrinsic value of stocks
'''
class CalculateValue(object):
    def __init__(self, data):
        self.data = data
    '''
     Discounted cash flow (DCF) is a valuation method used to estimate the value 
     of an investment based on its future cash flows. DCF analysis attempts to 
     figure out the value of an investment today, based on projections of 
     how much money it will generate in the future.
     param: years = valuation period to be considered
            rate_discount = rate of return that wants to make
            tv_multiplier = terminal value multiplier 
                           (for high quality businesses recommended: 15 and
                            10 for lower quality businesses)
     return: enterprise_value = EV calculated to achieve the set goal
             dcf_value = Stock price calculated to achieve the set goal         
    '''
    def simple_dcf_valuation(self, years, rate_discount, tv_multiplier):
        #free cash flow
        fcf = self.data['Market Cap(B)']/self.data['P/FCF']
          
        growth_1 = self.data['EPS next 5Y %'] #growth of first 5 years
        growth_2 = self.data['EPS next 5Y %']*0.7 #growth of remaining years
          
        compound_list = []
        pv_list = []
        compound_list.append(fcf)
        pv = fcf/(1+rate_discount)
        pv_list.append(pv)
          
        for year in range(2, years//2 + 1):
            new_fcf = fcf + fcf*growth_1
            fcf = new_fcf
            pv = fcf/(1 + rate_discount)**year
            compound_list.append(fcf)
            pv_list.append(pv)
        
        for year in range(years//2 + 1, years + 1):
            new_fcf = fcf + fcf*growth_2
            fcf = new_fcf
            pv = fcf/(1 + rate_discount)**year
            compound_list.append(fcf)
            pv_list.append(pv)
        
        terminal_value = compound_list[-1]*tv_multiplier
        tv_pv = terminal_value/(1 + rate_discount)**years
        pv_list.append(tv_pv)
        enterprise_value = sum(pv_list)
        dcf_value = enterprise_value/self.data['Shs Outstand(M)']*1000 #*1000 because EV(billion)/Shs Outstd(million)
        return enterprise_value, dcf_value
        
    '''
    'Sticker Price'(Fair Value of a Stock) and 'Margin of Safety' valuation method
     param: years = valuation period to be considered
            rate_return = wanted rate of return out of the investment
            margin_safety = is the risk tolerance of the investment. 
                            i.e) a margin of 0.5 means I'll buy the stock
                                 wen priced at 0.5 or less of their 
                                 intrinsic value(fair value)
     return: fairvalue = sticker price or intrinsic value of the stock
             buyvalue = buying price of stock 
    '''
    def sp_valuation(self, years, rate_return, margin_safety):
        eps = self.data['EPS (ttm)']
        growth = self.data['EPS next 5Y %']
        pe = 2*growth*100
        
        compound_list = []
        compound_list.append(eps) #year 1)
        for year in range(years-1):
            new_eps = eps + eps*growth
            eps = new_eps
            compound_list.append(eps)
        values = []
        value = compound_list[-1]*pe
        values.append(value)
        for year in range(years-1):
            value = value/(1 + rate_return)
            values.append(value)
        
        fairvalue = values[-1]
        buyvalue = fairvalue*margin_safety
        return fairvalue, buyvalue

In [22]:
intrinsic_value = CalculateValue(data)
#'Sticker Price'(Fair Value of a Stock) and 'Margin of Safety' valuation method
fairvalue_1, buyvalue_1 = intrinsic_value.sp_valuation(years=10, rate_return=0.15, margin_safety=0.6666)
fairvalue_2, buyvalue_2 = intrinsic_value.sp_valuation(years=10, rate_return=0.15, margin_safety=0.5)
#Discounted cash flow valuation method
enterprise_value_1, dcf_value_1 = intrinsic_value.simple_dcf_valuation(years=10, rate_discount=0.1, tv_multiplier=10)
enterprise_value_2, dcf_value_2 = intrinsic_value.simple_dcf_valuation(years=10, rate_discount=0.1, tv_multiplier=15)

valuation = pd.DataFrame({'DCF EV(B) (tv=10)': enterprise_value_1, 
                          'DCF Value (tv=10': dcf_value_1,
                          'DCF EV(B) (tv=15)': enterprise_value_2, 
                          'DCF Value (tv=15)': dcf_value_2,
                          'SP Fair Value': fairvalue_1, #same as fairvalue_1, so I'll print just one of them
                          'SP Buy Value (ms=0.66)': buyvalue_1,
                          'SP Buy Value (ms=0.5)': buyvalue_2})
results = pd.concat([valuation, data], axis=1)

In [23]:
results

,DCF EV(B) (tv=10),DCF Value (tv=10,DCF EV(B) (tv=15),DCF Value (tv=15),SP Fair Value,SP Buy Value (ms=0.66),SP Buy Value (ms=0.5),Market Cap(B),Price,Target Price,...,Insider Own %,ROE %,ROI %,ROA %,Profit Margin %,Shs Outstand(M),RSI (14),Beta,52W Range Low,52W Range High
V,215.862497,109.021463,273.682833,138.223653,188.980941,125.974695,94.490470,447.97,212.30,275.42,...,0.0013,0.308,0.202,0.128,0.484,1980.00,42.61,0.94,192.81,252.67
RDS-A,293.264574,75.389351,360.533511,92.682136,10.637488,7.090949,5.318744,177.55,45.28,59.08,...,0.0020,0.028,-0.072,0.012,0.021,3890.00,47.04,0.82,30.80,50.75
EURN,NaN,NaN,NaN,NaN,-1.793845,-1.195777,-0.896923,2.03,10.08,12.05,...,0.0024,NaN,NaN,NaN,NaN,201.68,46.71,NaN,7.55,11.20
V,215.862497,109.021463,273.682833,138.223653,188.980941,125.974695,94.490470,447.97,212.30,275.42,...,0.0013,0.308,0.202,0.128,0.484,1980.00,42.61,0.94,192.81,252.67
PYPL,140.479806,120.068210,179.266185,153.218961,256.849115,171.215620,128.424557,242.20,212.54,288.75,...,0.0010,0.239,0.084,0.068,0.201,1170.00,33.04,1.13,183.54,310.16
ABBV,158.285349,89.426751,192.135153,108.550934,15.869937,10.578900,7.934969,205.96,116.84,127.78,...,0.0012,0.564,0.127,0.050,0.135,1770.00,65.59,0.79,97.40,121.53
ADBE,167.163890,350.448406,212.003282,444.451324,555.952361,370.597844,277.976180,307.01,659.73,706.37,...,0.0031,0.424,0.306,0.231,0.387,477.00,58.65,1.07,420.78,677.76
AMGN,52.183451,92.034305,63.659760,112.274709,53.594026,35.725778,26.797013,116.92,207.40,244.81,...,0.0010,0.637,0.195,0.090,0.218,567.00,42.64,0.61,200.47,276.69
BABA,470.558704,172.999524,583.265732,214.435931,121.645676,81.089008,60.822838,455.51,166.54,243.13,...,0.1018,0.160,0.014,0.091,0.192,2720.00,51.60,0.88,138.43,280.61
CVS,166.454141,126.101622,202.846074,153.671268,29.320247,19.544877,14.660124,123.66,93.49,103.90,...,0.0010,0.105,0.074,0.033,0.027,1320.00,64.39,0.81,65.18,96.57


In [28]:
import json
from datetime import datetime
with open(f"{datetime.today().strftime('%Y-%m-%d')}.json", 'w', encoding='utf-8') as f:
    json.dump(new_dict, f, ensure_ascii=False, indent=4)